# Project 2 - Apache Cassandra Data Modeling

_**NOTE**: To better understand what has been done, please read first the README that is inside the project folder in Github:
**udacity-de/apache-cassandra-data-modeling/README.md**_

# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# import python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from os.path import dirname

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# get the current folder and subfolder event data
filepath = dirname(dirname(os.getcwd())) + '/data/event_data'

# collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

# check number of files
print(len(file_path_list))

/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/src/notebooks
['/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-15-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-22-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-09-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-18-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-04-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-01-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de/apache-cassandra-data-modeling/data/event_data/2018-11-27-events.csv', '/Users/sergiodacosta/Documents/_github/udacity-de

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading the csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skip header row
        next(csvreader)
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

file = 'event_datafile_new.csv'
event_datafile_path = dirname(dirname(os.getcwd())) + '/data/event_data/' + file
   
with open(event_datafile_path, 'w', encoding='utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open(event_datafile_path, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


-------

# Part II. Apache Cassandra Data Modeling (Coding)

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

![Data](./images/data-file.png)

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# this should make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster

# make a connection to a Cassandra instance - local machine 
cluster = Cluster(['127.0.0.1'])

# establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# create a new Keyspace 
create_keyspace_query = """
        CREATE KEYSPACE IF NOT EXISTS music_app_history
        WITH REPLICATION =
        { 'class': 'SimpleStrategy', 'replication_factor' : 1 }
    """

try:
    session.execute(create_keyspace_query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# set keyspace to the keyspace specified above
session.set_keyspace('music_app_history')

### Now we need to create tables to run the following queries (with Apache Cassandra we model the database tables on the queries we want to run)

## Create queries to ask the following three questions of the data
* QUESTION 1 - Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4


* QUESTION 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


* QUESTION 3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [8]:
# load data from csv to pandas dataframe
df_event_datafile = pd.read_csv(event_datafile_path, encoding='utf8')

In [9]:
df_event_datafile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6820 entries, 0 to 6819
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   artist         6820 non-null   object 
 1   firstName      6820 non-null   object 
 2   gender         6820 non-null   object 
 3   itemInSession  6820 non-null   int64  
 4   lastName       6820 non-null   object 
 5   length         6820 non-null   float64
 6   level          6820 non-null   object 
 7   location       6820 non-null   object 
 8   sessionId      6820 non-null   int64  
 9   song           6820 non-null   object 
 10  userId         6820 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 586.2+ KB


In [10]:
## creation of a new field named user_name where the user's full name is stored (first name + last name).
df_event_datafile['user_name'] = df_event_datafile.firstName + ' ' + df_event_datafile.lastName

In [11]:
df_event_datafile.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId,user_name
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26,Ryan Smith
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26,Ryan Smith


----

## Answers to the Questions


### QUESTION 1 - Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

* Table structure as: 
        * Column 1 = session_id
        * Column 2 = item_in_session
        * Column 3 = artist
        * Column 4 = song
        * Column 5 = length
        * Primary key = (session_id, item_in_session)

In [12]:
## CREATE TABLE: create table for query 1
query  =  """CREATE TABLE IF NOT EXISTS song_in_session_history(session_id INT, item_in_session INT, artist TEXT, song TEXT, length FLOAT, \
           PRIMARY KEY(session_id, item_in_session))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [13]:
## LOAD DATA: Insert data into table for query 1
for i, row in df_event_datafile.iterrows():
    query  = "INSERT INTO song_in_session_history (session_id, item_in_session, artist, song, length)"
    query += "VALUES(%s, %s, %s, %s, %s)"
    session.execute(query, (row.sessionId, row.itemInSession, row.artist, row.song, row.length))                    

#### SELECT to verify that the data have been inserted into table

* To exploit the results, I also used a cassandra web user interface to analyse the data.


* _The specific field names are used to retrieve data. This is also a very important point when we are accessing a very huge database. Doing SELECT * can be very costly in terms of reading._


* Fields name: **session_id, item_in_session, artist, song, length**

In [14]:
## SELECT statement for query 1
query = " SELECT session_id, item_in_session, artist, song, length \
          FROM song_in_session_history \
          WHERE session_id = 338  \
            AND item_in_session = 4 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## create new dataframe for query 1
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['session_id', 'item_in_session', 'artist', 'song', 'length'])

,session_id,item_in_session,artist,song,length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.307312


![Data](./images/cassandra-web-query-1.png)

------

### QUESTION 2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

* Table structure as: 
        * Column 1 = user_id
        * Column 2 = session_id
        * Column 3 = item_in_session
        * Column 4 = artist
        * Column 5 = song
        * Column 6 = user_name
        * Primary key = (user_id, session_id) with clustering column item_in_session

In [15]:
## CREATE TABLE: create table for query 2
query  =  """CREATE TABLE IF NOT EXISTS songs_in_user_history( user_id INT, session_id INT, item_in_session INT, artist TEXT, song TEXT, user_name TEXT, \
           PRIMARY KEY((user_id, session_id), item_in_session))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [16]:
## LOAD DATA: Insert data into table for query 2
for i, row in df_event_datafile.iterrows():
    query  = "INSERT INTO songs_in_user_history (user_id, session_id, item_in_session, artist, song, user_name)"
    query += "VALUES(%s, %s, %s, %s, %s, %s)"
    session.execute(query, (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, row.user_name))

#### SELECT to verify that the data have been inserted into table

* To exploit the results, I also used a cassandra web user interface to analyse the data.


* _The specific field names are used to retrieve data. This is also a very important point when we are accessing a very huge database. Doing SELECT * can be very costly in terms of reading._


* Fields name: **user_id, session_id, item_in_session, artist, song, user_name**

In [17]:
## SELECT statement for query 2
query = "SELECT user_id, session_id, item_in_session, artist, song, user_name \
         FROM songs_in_user_history \
         WHERE user_id = 10 \
            AND session_id = 182 \
        "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## create new dataframe for query 2
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['user_id', 'session_id', 'item_in_session', 'artist', 'song', 'user_name'])

,user_id,session_id,item_in_session,artist,song,user_name
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


![Data](./images/cassandra-web-query-2.png)

------

### QUESTION 3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

* Table structure as: 
        * Column 1 = song
        * Column 2 = user_id
        * Column 3 = user_name
        * Primary key = (song, user_id)

In [18]:
## CREATE TABLE: create table for query 3
query  =  """CREATE TABLE IF NOT EXISTS user_in_song_history (song TEXT, user_id INT, user_name TEXT, \
           PRIMARY KEY(song, user_id))"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [19]:
## LOAD DATA: Insert data into table for query 3
for i, row in df_event_datafile.iterrows():
    query  = "INSERT INTO user_in_song_history (song, user_id, user_name)"
    query += "VALUES(%s, %s, %s)"
    session.execute(query, (row.song, row.userId, row.user_name))

#### SELECT to verify that the data have been inserted into table

* To exploit the results, I also used a cassandra web user interface to analyse the data.


* _The specific field names are used to retrieve data. This is also a very important point when we are accessing a very huge database. Doing SELECT * can be very costly in terms of reading._


* Fields name: **song, user_id, user_name**

In [20]:
## SELECT statement for query 3
query = "SELECT song, user_id, user_name \
         FROM user_in_song_history \
         WHERE song = 'All Hands Against His Own' \
        "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

## create new dataframe for query 3
data = list(map(lambda row: list(row), rows))
pd.DataFrame(data, columns = ['song', 'user_id', 'user_name'])

,song,user_id,user_name
0,All Hands Against His Own,29,Jacqueline Lynch
1,All Hands Against His Own,80,Tegan Levine
2,All Hands Against His Own,95,Sara Johnson


![Data](./images/cassandra-web-query-3.png)

--------

### Drop the tables before closing out the sessions

In [21]:
session.execute("DROP TABLE IF EXISTS songs_in_user_history")

In [22]:
session.execute("DROP TABLE IF EXISTS songs_in_user_history")

In [23]:
session.execute("DROP TABLE IF EXISTS user_in_song_history")

------

### Drop the KEYSPACE before closing out the sessions

In [24]:
session.execute("DROP KEYSPACE IF EXISTS music_app_history")

--------

### Close the session and cluster connection¶

In [25]:
session.shutdown()
cluster.shutdown()

------------

### Remove the generated csv file

In [26]:
# Remove a file
os.remove(event_datafile_path) 

---------

-------